# Preprocessing Bureau balance

In [9]:
from pathlib import Path
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent   # notebooks/01_data_preparation -> projet root

DATA_RAW       = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN     = PROJECT_ROOT / "data" / "clean"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_CLEAN.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT =", PROJECT_ROOT)
print("DATA_RAW     =", DATA_RAW)

bureau_bal = pd.read_csv(DATA_RAW / "bureau_balance.csv")
print("bureau_balance:", bureau_bal.shape)
bureau_bal.head()

PROJECT_ROOT = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2
DATA_RAW     = c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\raw
bureau_balance: (27299925, 3)


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


# Bureau balanced

La table bureau_balance contient l’historique mensuel des crédits externes. Elle fournit deux informations clés :

- MONTHS_BALANCE, qui indique la profondeur temporelle de l’historique ;
- STATUS, qui décrit le statut mensuel du crédit (paiement normal, retard, fermeture, statut inconnu).

Comme un crédit peut avoir plusieurs mois d’historique, la table est agrégée au niveau SK_ID_BUREAU.

Les agrégations incluent :

- des statistiques temporelles : profondeur historique, mois le plus récent, nombre total de mois ;
- des comptages de statuts : nombre de mois payés, retards légers, retards sévères, mois fermés, statuts inconnus ;
- des ratios comportementaux : proportion de retards, proportion de retards sévères, proportion de mois fermés ;
- des fréquences normalisées : distribution des statuts sur l’ensemble de l’historique.

Ces agrégations permettent de transformer un historique mensuel complexe en indicateurs synthétiques et exploitables, décrivant la régularité, la stabilité et la qualité du comportement de crédit du client.

In [10]:
bureau_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


## Nettoyage

In [11]:

bb = bureau_bal.copy()

# Types
bb["SK_ID_BUREAU"] = bb["SK_ID_BUREAU"].astype("int32")
bb["MONTHS_BALANCE"] = bb["MONTHS_BALANCE"].astype("int16")

# STATUS : strip + normalisation
bb["STATUS"] = bb["STATUS"].astype("string").str.strip()

# Remplacer X -> UNKNOWN (mieux que NaN pour get_dummies et agrégats)
bb["STATUS"] = bb["STATUS"].replace({"X": "UNKNOWN"})

# Optionnel : si jamais tu as des NA réels
bb["STATUS"] = bb["STATUS"].fillna("UNKNOWN")

bb["BB_STATUS_UNKNOWN_FLAG"] = (bb["STATUS"] == "UNKNOWN").astype("int8")

bb[["SK_ID_BUREAU", "MONTHS_BALANCE", "STATUS"]].head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


## Aggrégats temporel

In [12]:

bb_time = bb.groupby("SK_ID_BUREAU", as_index=True).agg(
    BB_MONTHS_MIN=("MONTHS_BALANCE", "min"),   # profondeur max (ex: -96)
    BB_MONTHS_MAX=("MONTHS_BALANCE", "max"),   # souvent 0
    BB_MONTHS_COUNT=("MONTHS_BALANCE", "count"),
    BB_MONTHS_NUNIQUE=("MONTHS_BALANCE", "nunique"),
)

# span d'historique (durée)
bb_time["BB_MONTHS_SPAN"] = bb_time["BB_MONTHS_MAX"] - bb_time["BB_MONTHS_MIN"]
bb_time.head()

,BB_MONTHS_MIN,BB_MONTHS_MAX,BB_MONTHS_COUNT,BB_MONTHS_NUNIQUE,BB_MONTHS_SPAN
SK_ID_BUREAU,,,,,
5001709,-96,0,97,97,96
5001710,-82,0,83,83,82
5001711,-3,0,4,4,3
5001712,-18,0,19,19,18
5001713,-21,0,22,22,21


## Counts de status + ratios

In [13]:

status_counts = (
    pd.get_dummies(bb["STATUS"], prefix="BB_STATUS")
    .assign(SK_ID_BUREAU=bb["SK_ID_BUREAU"])
    .groupby("SK_ID_BUREAU", as_index=True)
    .sum()
)

# Ratios (safe)
den = bb_time["BB_MONTHS_COUNT"].replace(0, np.nan)

status_ratios = status_counts.div(den, axis=0)
status_ratios = status_ratios.add_suffix("_RATIO")

status_counts.head(), status_ratios.head()

(              BB_STATUS_0  BB_STATUS_1  BB_STATUS_2  BB_STATUS_3  BB_STATUS_4  BB_STATUS_5  BB_STATUS_C  BB_STATUS_UNKNOWN
 SK_ID_BUREAU                                                                                                              
 5001709                 0            0            0            0            0            0           86                 11
 5001710                 5            0            0            0            0            0           48                 30
 5001711                 3            0            0            0            0            0            0                  1
 5001712                10            0            0            0            0            0            9                  0
 5001713                 0            0            0            0            0            0            0                 22,
               BB_STATUS_0_RATIO  BB_STATUS_1_RATIO  BB_STATUS_2_RATIO  BB_STATUS_3_RATIO  BB_STATUS_4_RATIO  BB_STATUS_5_RATIO  BB

## Features métier "retards"

In [14]:

late_status = {"1", "2", "3", "4", "5"}
severe_status = {"3", "4", "5"}

bb["BB_LATE_FLAG"] = bb["STATUS"].isin(late_status).astype("int8")
bb["BB_SEVERE_LATE_FLAG"] = bb["STATUS"].isin(severe_status).astype("int8")

bb_late = bb.groupby("SK_ID_BUREAU", as_index=True).agg(
    BB_LATE_COUNT=("BB_LATE_FLAG", "sum"),
    BB_SEVERE_LATE_COUNT=("BB_SEVERE_LATE_FLAG", "sum"),
)

bb_late["BB_LATE_RATIO"] = bb_late["BB_LATE_COUNT"] / den
bb_late["BB_SEVERE_LATE_RATIO"] = bb_late["BB_SEVERE_LATE_COUNT"] / den

bb_late = bb_late.fillna(0)
bb_late.head()

,BB_LATE_COUNT,BB_SEVERE_LATE_COUNT,BB_LATE_RATIO,BB_SEVERE_LATE_RATIO
SK_ID_BUREAU,,,,
5001709,0,0,0.0,0.0
5001710,0,0,0.0,0.0
5001711,0,0,0.0,0.0
5001712,0,0,0.0,0.0
5001713,0,0,0.0,0.0


## Fusion Finale + export

In [15]:

bb_agg = (
    bb_time
    .join(status_counts, how="left")
    .join(status_ratios, how="left")
    .join(bb_late, how="left")
    .reset_index()
)

# Remplacer NaN (rare) par 0 sur les features dérivées
# (BB_MONTHS_* ne devraient pas être NaN)
cols_fill0 = [c for c in bb_agg.columns if c.startswith("BB_STATUS_") or c.startswith("BB_LATE_")]
bb_agg[cols_fill0] = bb_agg[cols_fill0].fillna(0)

print("bb_agg:", bb_agg.shape)
bb_agg.head()

bb_agg: (817395, 26)


,SK_ID_BUREAU,BB_MONTHS_MIN,BB_MONTHS_MAX,BB_MONTHS_COUNT,BB_MONTHS_NUNIQUE,BB_MONTHS_SPAN,BB_STATUS_0,BB_STATUS_1,BB_STATUS_2,BB_STATUS_3,BB_STATUS_4,BB_STATUS_5,BB_STATUS_C,BB_STATUS_UNKNOWN,BB_STATUS_0_RATIO,BB_STATUS_1_RATIO,BB_STATUS_2_RATIO,BB_STATUS_3_RATIO,BB_STATUS_4_RATIO,BB_STATUS_5_RATIO,BB_STATUS_C_RATIO,BB_STATUS_UNKNOWN_RATIO,BB_LATE_COUNT,BB_SEVERE_LATE_COUNT,BB_LATE_RATIO,BB_SEVERE_LATE_RATIO
0,5001709,-96,0,97,97,96,0,0,0,0,0,0,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,0,0,0.0,0.0
1,5001710,-82,0,83,83,82,5,0,0,0,0,0,48,30,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,0,0,0.0,0.0
2,5001711,-3,0,4,4,3,3,0,0,0,0,0,0,1,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0,0,0.0,0.0
3,5001712,-18,0,19,19,18,10,0,0,0,0,0,9,0,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.0,0,0,0.0,0.0
4,5001713,-21,0,22,22,21,0,0,0,0,0,0,0,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.0,0.0


In [16]:

OUT_PATH = DATA_PROCESSED / "bureau_balance_agg.csv"
bb_agg.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

Saved: c:\Users\yoann\Documents\open classrooms\projet 6\pret a depenser v2\data\processed\bureau_balance_agg.csv
